* [pygazer](https://www.pygaze.org/about/) - аналог webgazer для python, устаревшая библиотека для python 2.7(пофиксить ошибки + запустить не получилось)
* [openCV-gaze-tracking](https://learnopencv.com/gaze-tracking/) - используют дополнительно ИК камеры для отслеживания глаз
* [camera-to-screen](https://github.com/pperle/gaze-tracking-pipeline/tree/main#) - openCV + pytorch, кажется, рабочий пайплайн: на видео с камеры выделяется лицо, глаза и [точки на лице](https://github.com/google-ai-edge/mediapipe/wiki/MediaPipe-Face-Mesh), подаются на вход сверточной нейросети(в данной работе используются предобученные слои VGG16 на imagenet), которая предсказывает 3D вектор направления взгляда(gaze vector), затем этот вектор проецируется на экран в координаты (x, y) (калибровка, например, с помощью полиномиальной регрессии), в конце в идеале добавить фильтрацию(например, фильтр Калмана), чтобы вз1гляд не дрожал на экране.
* [Webcam-based gaze estimation for computer screen](https://gazerecorder.com/wp-content/uploads/2024/04/frobt-11-1369566-1.pdf) - прочитал основную часть статьи, этот подход кажется проще, чем предыдущий, хотя они очень похожи. В данной статье берут готовую модель "gaze-estimation-adas-0002" из [библиотеки](https://github.com/openvinotoolkit/openvino) для CV от Intel, которая на входе получает обрезанную часть картинки с правым глазом, с левым глазом и вектор положения головы(3 угла), а на выходе дает вектор направления взгляда единичной длины. Далее в статье показывают, как с помощью матрицы однородного преобразования(афинное без растягиваний), проецируется вектор направления взгляда в точку на экране, где расстояние от глаз до экрана оценивается с помощью регрессии(данные берутся на этапе калибровки для каждого отдельно взятого человека, в статье использовали 4 точки по разным углам монитора).

In [18]:
import openvino as ov
core = ov.Core()

In [19]:
# to download models VPN needed
# pip install openvino-dev
# omz_downloader --name <name-of-the-model> 

FACE_MODEL_PTH             = r"../../intel/face-detection-0200/FP32/face-detection-0200.xml"
HEAD_POSE_MODEL_PTH        = r"../../intel/head-pose-estimation-adas-0001/FP32/head-pose-estimation-adas-0001.xml"
FACIAL_LANDMARKS_MODEL_PTH = r"../../intel/landmarks-regression-retail-0009/FP32/landmarks-regression-retail-0009.xml"
GAZE_ESTIMATION_MODEL_PTH  = r".../../intel/gaze-estimation-adas-0002/FP32/gaze-estimation-adas-0002.xml"

In [20]:
face_detection_model = core.read_model(FACE_MODEL_PTH)
compiled_face_detection_model = core.compile_model(face_detection_model, device_name="CPU")

In [21]:
input_shape = compiled_face_detection_model.input(0).shape
model_h, model_w = input_shape[2], input_shape[3]

In [ ]:
import cv2
import numpy as np

INPUT_IMG_PTH  = r"images/img.png"
OUTPUT_IMG_PTH = r"images/result.png"

img = cv2.imread(INPUT_IMG_PTH)
initial_height, initial_width = img.shape[:2]

resized = cv2.resize(img, (model_w, model_h))
input_image = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
input_image = np.transpose(input_image, (2, 0, 1)) # C W H
input_image = np.expand_dims(input_image, axis=0) # N C W H

results = compiled_face_detection_model(input_image)
output = results[compiled_face_detection_model.output(0)].squeeze()

In [23]:
THRESHOLD = 0.5

faces_count = 0
cropped_faces = []
for _, _, confidence, x_min, y_min, x_max, y_max in output:
    
    if confidence < THRESHOLD:
        break
    
    x_min, y_min = int(x_min * initial_width), int(y_min * initial_height)
    x_max, y_max = int(x_max * initial_width), int(y_max * initial_height)
    
    color = (0, 255, 0) # green, BGR
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, thickness=2)
    cropped_face = img[y_min:y_max, x_min:x_max]
    cropped_faces.append(cropped_face)
    
    label = f"Face: {confidence:.2f}"
    cv2.putText(img, label, (x_min, y_min - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    faces_count += 1

print(f"Faces found: {faces_count}")
cv2.imwrite(OUTPUT_IMG_PTH, img)

Faces found: 1


True

In [24]:
facial_landmarks_model = core.read_model(FACIAL_LANDMARKS_MODEL_PTH)
compiled_landmarks_model = core.compile_model(facial_landmarks_model, "CPU")

In [34]:
face1 = cropped_faces[0]
initial_height, initial_width = face1.shape[:2]

input_shape = compiled_landmarks_model.input(0).shape
model_h, model_w = input_shape[2], input_shape[3]

input_image = np.resize(face1, (model_h, model_w, 3))
input_image = np.transpose(input_image, (2, 0, 1))
input_image = np.expand_dims(input_image, axis=0)
results = compiled_landmarks_model(input_image)
output = results[compiled_landmarks_model.output(0)].squeeze()

In [36]:
for i in range(0, len(output), 2):
    x, y = int(output[i] * initial_width), int(output[i+1] * initial_height)
    
    color = (0, 0, 255)
    print(x, y)
    cv2.circle(face1, (x, y), 1, color, 10)

cv2.imwrite(OUTPUT_IMG_PTH, face1)

26 76
119 84
48 111
19 152
112 160


True